# Upload to Cloud Object Storage COS HTTP adapter

Create a HTTP service to upload a file to any S3 compliant Cloud Object Storage

In [2]:
import os
#os.environ['create_image']='True'

In [3]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM registry.access.redhat.com/ubi8/python-39
    RUN pip install ipython nbformat aiobotocore botocore s3fs flask
    ADD upload-to-cos.ipynb /
    ADD upload-to-cos-http-adapter.ipynb /
    ENTRYPOINT ["ipython","/upload-to-cos-http-adapter.ipynb"]
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t upload-to-cos-http-adapter .
    !docker tag upload-to-cos-http-adapter romeokienzler/upload-to-cos-http-adapter
    !docker push romeokienzler/upload-to-cos-http-adapter
elif bool(os.environ.get('install_dependencies',False)):
    !pip install aiobotocore botocore s3fs

Sending build context to Docker daemon  39.94kB
Step 1/5 : FROM registry.access.redhat.com/ubi8/python-39
 ---> a531ae76755e
Step 2/5 : RUN pip install ipython nbformat aiobotocore botocore s3fs flask
 ---> Using cache
 ---> f349e56a05a3
Step 3/5 : ADD upload-to-cos.ipynb /
 ---> 8a007fd5de44
Step 4/5 : ADD upload-to-cos-http-adapter.ipynb /
 ---> cf63d3c9918f
Step 5/5 : ENTRYPOINT ["ipython","/upload-to-cos-http-adapter.ipynb"]
 ---> Running in 276421fbdf6a
Removing intermediate container 276421fbdf6a
 ---> 30367437d548
Successfully built 30367437d548
Successfully tagged upload-to-cos-http-adapter:latest
Using default tag: latest
The push refers to repository [docker.io/romeokienzler/upload-to-cos-http-adapter]

9fa60caf: Preparing 
a6c2215a: Preparing 
b4330ba2: Preparing 
2e0f4ef5: Preparing 
276847a2: Preparing 
534f4e1b: Preparing 
c926eef9: Preparing 
9fa60caf: Pushed lready exists 4kBlatest: digest: sha256:4ece79573cbbb8988f04fd5a10ada81b5f1db70590bced8eb29f43616f76cad4 size: 20

In [ ]:
from flask import request
from flask import Flask
import time
import json

In [ ]:
app = Flask(__name__)

@app.route('/', methods=['POST'])
def index():
    message = json.dumps(request.get_json())
    print(message)
    now = time.time()
    print(now)
    with open('/tmp/source.json','w') as file:
        file.write(message)
    !ipython /upload-to-cos.ipynb source_file="source.json" destination_file=$now".json"
    return ''

app.run(host='0.0.0.0', port=8080)